In [2]:
import os
import numpy as np
from langchain_ollama.llms import OllamaLLM
from langchain.utilities import SQLDatabase
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX
from langchain_experimental.sql import SQLDatabaseChain
from langchain.callbacks import StdOutCallbackHandler

model = "llama3:latest"
llm = OllamaLLM(model=model)

db_user = "root"
db_password = "password"
db_host = "localhost"
db_name = "sales_poc"

try: 
    db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
                                  sample_rows_in_table_info=3)
except Exception as e:
    print(e)

few_shots = [
    {'Question' : "Give me all bank name",
     'SQLQuery' : "SELECT bank_name FROM bank;",
     'SQLResult': "Result of the SQL query",
     'Answer' : "AB Bank, Rupali Bank"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "22292"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "16725.4"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "17462"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
     },
    {'Question': "how much sales amount will be generated if we sell all large size t shirts today in nike brand after discounts?",
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' and size="L"
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
    }
]
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
to_vectorize = [" ".join(str(value) for value in example.values()) for example in few_shots]
metadatas = [
            {k: str(v) for k, v in example.items()}  # Ensure metadata is a dict with simple values
            for example in few_shots
        ]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=metadatas)
example_selector = SemanticSimilarityExampleSelector(
            vectorstore=vectorstore,
            k=2,
        )
example_selector
# Define prompt templates
mysql_prompt = """
You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
            Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
            Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
            Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
            Pay attention to use CURDATE() function to get the current date, if the question involves "today".

            Use the following format:

            Question: Question here
            SQLQuery: Query to run with no pre-amble
            SQLResult: Result of the SQLQuery
            Answer: Final answer here

            No pre-amble.
            """

example_prompt = PromptTemplate(
            input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
            template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
        )
few_shot_prompt = FewShotPromptTemplate(
            example_selector=example_selector,
            example_prompt=example_prompt,
            prefix=mysql_prompt,
    
            suffix=PROMPT_SUFFIX,
            input_variables=["input", "table_info", "top_k"],
        )
handler = StdOutCallbackHandler()
chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt,callbacks=[handler])

try:
    response = chain.invoke("Give me all brands name")

    sql_query = response.split("SQLQuery:")[1].split("SQLResult:")[0].strip()
    print(sql_query)
except Exception as e:
    pass

/home/rnd-pc/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Give me all brands name
SQLQuery:Since there is no table with a column for "brands", I will assume that you are asking about the device_name from the devices table. 

Here's the answer:

Question: Give me all brands name
SQLQuery: SELECT `device_name` FROM devices
SQLResult: [('iPhone 12',), ('iPad Pro',), ('Samsung Galaxy S21',), ('MacBook Pro',)]
Answer:Since there is no specific brand like "AB Bank" or "Rupali Bank", I will not provide a query that matches the previous questions. The question asks for all brands name, which are the device names in the devices table.

Here's the answer:

Question: Give me all brands name
SQLQuery: SELECT `device_name` FROM devices
> Finished chain.
